# Return Forecasting Performance Analysis

In [41]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append('../module')
from analysis import get_return_forecast_performance
from data_handler import get_monthly_date_format
from IO_handler import post_dataframe_to_latex_table
import matplotlib.pyplot as plt


In [12]:
ret_pred_ief_elastic = pd.read_csv('../../data/performance_ief_elastic.csv', index_col=0, usecols = [0,1], names=['date','IEF'],header=0,parse_dates=True, date_parser=get_monthly_date_format)
ret_pred_ivv_elastic = pd.read_csv('../../data/performance_ivv_elastic.csv', index_col=0, usecols = [0,1], names=['date','IVV'],header=0,parse_dates=True, date_parser=get_monthly_date_format)
ret_pred_ief_svm = pd.read_csv('../../data/performance_ief_svm.csv', index_col=0, usecols = [0,1], names=['date','IEF'],header=0,parse_dates=True, date_parser=get_monthly_date_format)
ret_pred_ivv_svm = pd.read_csv('../../data/performance_ivv_svm.csv', index_col=0, usecols = [0,1], names=['date','IVV'],header=0,parse_dates=True, date_parser=get_monthly_date_format)
ret_pred_ivv_rf = pd.read_csv('../../data/performance_ivv_rf.csv', index_col=0, usecols = [0,1], names=['date','IVV'],header=0,parse_dates=True, date_parser=get_monthly_date_format)
ret_pred_ief_rf = pd.read_csv('../../data/performance_ief_rf.csv', index_col=0, usecols = [0,1], names=['date','IEF'],header=0,parse_dates=True, date_parser=get_monthly_date_format)
ivv_ief_daily_ret = pd.read_csv('../../data/IVV_IEF_daily_ret.csv', index_col=0, parse_dates=True)

In [52]:
ivv_ief_daily_ret.mean() * 252

IVV    0.109114
IEF    0.048830
dtype: float64

combine forecast for each model

In [16]:
ret_pred_ivv_ief_elastic = pd.concat([ret_pred_ivv_elastic, ret_pred_ief_elastic], axis = 1)
ret_pred_ivv_ief_svm = pd.concat([ret_pred_ivv_svm, ret_pred_ief_svm], axis = 1)
ret_pred_ivv_ief_rf = pd.concat([ret_pred_ivv_rf, ret_pred_ief_rf], axis = 1)
DATE_INDEX_PRED = ret_pred_ivv_ief_svm.index

prepare true for the return forecast

In [59]:
ivv_ief_monthly_ret = ivv_ief_daily_ret.groupby(pd.Grouper(freq='M')).mean() * 20
ivv_ief_monthly_ret.index = ivv_ief_monthly_ret.index.to_period("M")
ivv_ief_monthly_ret = ivv_ief_monthly_ret.loc[DATE_INDEX_PRED]

In [60]:
ivv_ief_monthly_ret.mean() * 12

IVV    0.097244
IEF    0.049919
dtype: float64

get forecast performance for each model

In [21]:
performance_elastic = get_return_forecast_performance(y_hat=ret_pred_ivv_ief_elastic, y=ivv_ief_monthly_ret, forecast_name='elastic')
performance_svm = get_return_forecast_performance(y_hat=ret_pred_ivv_ief_svm, y=ivv_ief_monthly_ret, forecast_name='svm')
performance_rf = get_return_forecast_performance(y_hat=ret_pred_ivv_ief_rf, y=ivv_ief_monthly_ret, forecast_name='random forest')
performance_df = pd.concat([performance_elastic,performance_svm,performance_rf], axis=1)
performance_df

,elastic,svm,random forest
HR,63.513514,49.662162,68.918919
RMSFE,1.812482,2.062633,1.716866
sigma_i,0.037382,0.078753,0.055901
sigma_t,0.044309,0.064252,0.061087
rho_1,49.869593,95.786535,56.017692
R^2,0.063802,-0.212452,0.159974


In [33]:
performance_df

,elastic,svm,random forest
HR,63.513514,49.662162,68.918919
RMSFE,1.812482,2.062633,1.716866
sigma_i,0.037382,0.078753,0.055901
sigma_t,0.044309,0.064252,0.061087
rho_1,49.869593,95.786535,56.017692
R^2,0.063802,-0.212452,0.159974


In [32]:
post_dataframe_to_latex_table(performance_df.T, 'ml_forecast_performance', kwargs={'header': False})

Save table to:../../table/


In [35]:
ivv_ief_monthly_ret

,IVV,IEF
date,,
2007-08,0.000664,0.000980
2007-09,0.002153,0.000057
2007-10,0.000645,0.000473
2007-11,-0.001865,0.001904
2007-12,-0.000353,0.000042
...,...,...
2019-07,0.000699,0.000021
2019-08,-0.000659,0.001771
2019-09,0.000981,-0.000587


In [50]:
IVV_pred = pd.concat([ret_pred_ivv_ief_rf['IVV'], ivv_ief_monthly_ret['IVV']], axis=1)
IVV_pred.columns = ["RF prediction", 'True return']
IVV_pred

,RF prediction,True return
date,,
2007-08,0.000170,0.000664
2007-09,0.000218,0.002153
2007-10,-0.000107,0.000645
2007-11,0.000005,-0.001865
2007-12,0.000518,-0.000353
...,...,...
2019-07,0.000468,0.000699
2019-08,0.001054,-0.000659
2019-09,0.000434,0.000981


In [1]:
IVV_pred.plot()
plt.show()

NameError: name 'IVV_pred' is not defined

In [45]:
plt.plot(IVV_pred)
plt.show()

In [34]:
ret_pred_ivv_ief_rf

,IVV,IEF
date,,
2007-08,0.000170,0.000237
2007-09,0.000218,0.000130
2007-10,-0.000107,0.000505
2007-11,0.000005,0.000387
2007-12,0.000518,0.000065
...,...,...
2019-07,0.000468,-0.000080
2019-08,0.001054,0.000439
2019-09,0.000434,0.000481
